# Problem 0: Soccer Guru

_Version 1.5_

Soccer season is on and teams need to start preparing for the World Cup 2022. We need your help as a **Soccer Guru** to analyse different statistics and come up with insights to help the teams prepare better.

This problem tests your understanding of Pandas and SQL concepts.

**Important note.** Due to a limitation in Vocareum's software stack, this notebook is set to use the Python 3.5 kernel (rather than a more up-to-date 3.6 or 3.7 kernel). If you are developing on your local machine and are using a different version of Python, you may need to adapt your solution before submitting to the autograder.


**Exercise 0** (0 points). Run the code cell below to load the data, which is a SQLite3 database containing results and fixtures of various soccer matches that have been played around the globe since 1980.

Observe that the code loads all rows from the table, `soccer_results`, contained in the database file, `prob0.db`.

> You do not need to do anything for this problem other than run the next two code cells and familiarize yourself with the resulting dataframe, which is stored in the variable `df`.

In [4]:
import sqlite3 as db
import pandas as pd
from datetime import datetime
from collections import defaultdict
disk_engine = db.connect('file:prob0.db?mode=ro', uri=True)

def load_data():
    df = pd.read_sql_query("SELECT * FROM soccer_results", disk_engine) 
    return df

In [5]:
# Test: Exercise 0 (exposed)
df = load_data()
assert df.shape[0] == 22851, "Row counts do not match. Try loading the data again"
assert df.shape[1] == 9, "You don't have all the columns. Try loading the data again"
print("\n(Passed!)")
df.head()


(Passed!)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,FALSE
1,1994-01-02,Ghana,Egypt,2,1,Friendly,Accra,Ghana,FALSE
2,1994-01-05,Mali,Burkina Faso,1,1,Friendly,Bamako,Mali,FALSE
3,1994-01-09,Mauritania,Mali,1,3,Friendly,Nouakchott,Mauritania,FALSE
4,1994-01-11,Thailand,Nigeria,1,1,Friendly,Bangkok,Thailand,FALSE


In [6]:
df['tournament'].unique()

array(['Friendly', 'CFU Caribbean Cup qualification',
       'United Arab Emirates Friendship Tournament',
       'Malta International Tournament', 'Lunar New Year Cup',
       'African Cup of Nations', 'CFU Caribbean Cup',
       'UEFA Euro qualification', 'Kirin Cup', 'FIFA World Cup',
       'Oceania Nations Cup qualification', 'Baltic Cup',
       'African Cup of Nations qualification', 'Gulf Cup',
       'Simba Tournament', 'CECAFA Cup', 'Confederations Cup',
       'Dynasty Cup', "King's Cup", 'Nehru Cup', 'SAFF Cup',
       'Copa Paz del Chaco', 'Korea Cup', 'USA Cup', 'Copa América',
       'Island Games', 'Merdeka Tournament', 'South Pacific Games',
       'UNCAF Cup', 'Oceania Nations Cup', 'Amílcar Cabral Cup',
       'Windward Islands Tournament', 'Gold Cup',
       'AFC Asian Cup qualification', 'FIFA World Cup qualification',
       'UEFA Euro', 'AFF Championship', 'AFC Asian Cup',
       'King Hassan II Tournament', 'Cyprus International Tournament',
       'Dunhill Cup'

Each row of this dataframe is a game, which is played between a "home team" (column `home_team`) and an "away team" (`away_team`). The number of goals scored by each team appears in the `home_score` and `away_score` columns, respectively.

**Exercise 1** (1 point): Write an **SQL query** find the ten (10) teams that have the highest average away-scores since the year 2000. Your query should satisfy the following criteria:

- It should return two columns:
    * `team`: The name of the team
    * `ave_goals`: The team's average number of goals **in "away" games.** An "away game" is one in which the team's name appars in `away_team` **and** the game takes place at a "non-neutral site" (`neutral` value equals `FALSE`).
- It should only include teams that have played **at least 30 away matches**.
- It should round the average goals value (`ave_goals`) to three decimal places.
- It should only return the top 10 teams in descending order by average away-goals.
- It should only consider games played since 2000 (including the year 2000).

Store your query string as the variable, `query_top10_away`, below. The test cell will run this query string against the input dataframe, `df`, defined above and return the result in a dataframe named `offensive_teams`. (See the test cell.)

> **Note.** The following exercises have hidden test cases and you'll be awarded full points for passing both the exposed and hidden test cases.

In [7]:
query_top10_away = r'''

### AY CODE

SELECT team, ave_goals
FROM 
(SELECT away_team as 'team',
round(avg(away_score),3) as 'ave_goals',
count(*) as 'count'
FROM soccer_results
WHERE date > '1999.12.31' AND
neutral = 'FALSE'
GROUP BY away_team
HAVING count >= 30
ORDER BY avg(away_score) DESC)
LIMIT 10'''  # Write your query here!

###
### YOUR CODE HERE
###











print(query_top10_away)


SELECT team, ave_goals
FROM 
(SELECT away_team as 'team',
round(avg(away_score),3) as 'ave_goals',
count(*) as 'count'
FROM soccer_results
WHERE date > '1999.12.31' AND
neutral = 'FALSE'
GROUP BY away_team
HAVING count >= 30
ORDER BY avg(away_score) DESC)
LIMIT 10


In [8]:
# Test: Exercise 1 (exposed)
offensive_teams = pd.read_sql_query(query_top10_away, disk_engine)
df_cols = offensive_teams.columns.tolist()
df_cols.sort()
desired_cols = ['team', 'ave_goals']
desired_cols.sort()
print(offensive_teams.head(10))
assert offensive_teams.shape[0] == 10, "Expected 10 rows but returned dataframe has {}".format(offensive_teams.shape[0])
assert offensive_teams.shape[1] == 2, "Expected 2 columns but returned dataframe has {}".format(offensive_teams.shape[1])
assert df_cols == desired_cols, "Column names should be: {}. Returned dataframe has: {}".format(desired_cols, df_cols)

tolerance = .001
team_4 = offensive_teams.iloc[3].team
team_4_ave = offensive_teams.iloc[3].ave_goals
desired_team_4_ave = 1.763
assert (team_4 == "England" and abs(team_4_ave - 1.763) <= .001), "Fourth entry is {} with average of {}. Got {} with average of {}".format("England", 1.76, team_4, team_4_ave)

print("\n(Passed!)")

           team  ave_goals
0       Germany      2.170
1        Brazil      2.010
2         Spain      1.927
3       England      1.763
4   Netherlands      1.742
5        France      1.639
6      Portugal      1.579
7     Argentina      1.560
8  Saudi Arabia      1.540
9       Denmark      1.534

(Passed!)


In [9]:
# Hidden test cell: exercise1_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Exercise 2** (2 points): Suppose we are now interested in the top 10 teams having the best goal **differential**, between the years 2012 and 2018 (both inclusive). A team's goal differential is the difference between the total number of goals it scored and the total number it conceded across all games (in the requested years).

Complete the function, `best_goal_differential()`, below, so that it returns a pandas dataframe containing the top 10 teams by goal differential, sorted in descending order of differential. The dataframe should have two columns: `team`, which holds the team's name, and `differential`, which holds its overall goal differential.

> As a sanity check, you should find the Brazil is the number one team, with a differential of 152 during the selected time period of 2012-2018 (inclusive). It should be the first row of the returned dataframe.

In [10]:
def best_goal_differential():
    ###
    ### YOUR CODE HERE
    ###
    import datetime
    df['date']=pd.to_datetime(df['date'])
    df_2 = df.loc[(df['date']>'2011.12.31')&(df['date']<'2019.1.1'),:]
    ht = df_2.groupby(['home_team'])[['home_score','away_score']].sum()
    ht['differential'] = ht['home_score']-ht['away_score']
    ht = ht.reset_index().rename(columns={'home_team':'team'})
    at = df_2.groupby(['away_team'])[['home_score','away_score']].sum()
    at['differential'] = at['away_score']-at['home_score']
    at = at.reset_index().rename(columns={'away_team':'team'})
    diff = ht[['team','differential']].append(at[['team','differential']])
    dfan = diff.groupby('team')['differential'].sum().reset_index().sort_values('differential',ascending=False)
    return dfan.iloc[:10,]

In [11]:
# Test: Exercise 2 (exposed)

diff_df = best_goal_differential()
df_cols = diff_df.columns.tolist()
df_cols.sort()
desired_cols = ['team', 'differential']
desired_cols.sort()

assert isinstance(diff_df, pd.DataFrame), "Dataframe object not returned"
assert diff_df.shape[0] == 10, "Expected 10 rows but returned dataframe has {}".format(diff_df.shape[0])
assert diff_df.shape[1] == 2, "Expected 2 columns but returned dataframe has {}".format(diff_df.shape[1])
assert df_cols == desired_cols, "Column names should be: {}. Returned dataframe has: {}".format(desired_cols, df_cols)

best_team = diff_df.iloc[0].team
best_diff = diff_df.iloc[0].differential
assert (best_team == "Brazil" and best_diff == 152), "{} has best differential of {}. Got team {} having best differential of {}".format("Brazil", 152, best_team, best_diff)

print("\n(Passed!)")


(Passed!)


In [12]:
# Hidden test cell: exercise2_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Exercise 3** (1 point). Complete the function, `determine_winners(game_df)`, below. It should determine the winner of each soccer game.

In particular, the function should take in a dataframe like `df` from above. It should return a new dataframe consisting of all the columns from that dataframe plus a new columnn called **`winner`**, holding the name of the winning team. If there is no winner for a particular game (i.e., the score is tied), then the `winner` column should containing the string, `'Draw'`. Lastly, the rows of the output should be in the same order as the input dataframe.

You can use any dataframe manipulation techniques you want for this question _(i.e., pandas methods or SQL queries, as you prefer)._

> You'll need the output dataframe from this exercise for the subsequent exercies, so don't skip this one!

In [13]:
def determine_winners(game_df):
    ###
    ### YOUR CODE HERE
    ###
    query = '''
    SELECT
    date,
    home_team,
    away_team,
    home_score,
    away_score,
    tournament,
    city,
    country,
    neutral,
    CASE 
    WHEN home_score > away_score THEN home_team
    WHEN home_score < away_score THEN away_team
    ELSE 'Draw'
    END AS 'winner'
    FROM soccer_results
    '''
    return pd.read_sql_query(query, disk_engine)

In [14]:
# Test: Exercise 3 (exposed)

game_df = pd.read_sql_query("SELECT * FROM soccer_results", disk_engine)
winners_df = determine_winners(game_df)

game_winner = winners_df.iloc[1].winner
assert game_winner == "Ghana", "Expected Ghana to be winner. Got {}".format(game_winner)

game_winner = winners_df.iloc[2].winner
assert game_winner == "Draw", "Match was Draw. Got {}".format(game_winner)

game_winner = winners_df.iloc[3].winner
assert game_winner == "Mali", "Expected Mali to be winner. Got {}".format(game_winner)

print("\n(Passed!)")


(Passed!)


In [15]:
# Hidden test cell: exercise3_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Exercise 4** (3 points): Given a team, its _home advantage ratio_ is the number of home games it has won divided by the number of home games it has played. For this exercise, we'll try to answer the question, how important is the home advantage in soccer? It's importance is factored into draws for competitions, for example, teams wanting to play at home the second leg of the matches of great importance such as tournament knockouts. (_This exercise has a pre-requisite of finishing Exercise 3 as we'll be using the results of the dataframe from that exercise in this one._)

Complete the function, `calc_home_advantage(winners_df)`, below, so that it returns the top 5 countries, among those that have played at least 50 **home** games, having the highest home advantage ratio. It should return a dataframe with two columns, **`team`** and **`ratio`**, holding the name of the team and its home advantage ratio, respectively. The ratio should be rounded to three decimal places. The rows should be sorted in descending order of ratio. If there are two teams with the same winning ratio, the teams should appear in alphabetical order by name.

> **Note 0.** As with our definition of away-games, a team plays a home game if it is the home team (`home_team`) **and** the field is non-neutral (i.e., `neutral` is `FALSE`).
>
> **Note 1.** You should find, for example, that Brazil is the number two team, with a home advantage ratio of 0.773.

In [16]:
def calc_home_advantage(winners_df):
    ###
    ### YOUR CODE HERE
    ###
    df = determine_winners(winners_df)
    df = df[df['neutral']=='FALSE']
    df['home_win'] = df.apply(lambda x: 1 if x['winner']==x['home_team'] else 0, axis=1)
    df = df.groupby('home_team')['home_win'].agg(['sum','count'])
    df['ratio'] = round(df['sum']/df['count'],3)
    df = df[df['count']>=50].sort_values('ratio',ascending=False).reset_index()
    df = df[['home_team','ratio']].rename(columns={'home_team':'team'})
    return df.iloc[:5,]

In [17]:
calc_home_advantage(df)

,team,ratio
0,Spain,0.800
1,Brazil,0.773
2,Iran,0.742
3,Cameroon,0.739
4,Egypt,0.724


In [18]:
# Test: Exercise 4 (exposed)
from IPython.display import display

win_perc = calc_home_advantage(winners_df)

print("The solution, according to you:")
display(win_perc)

df_cols = win_perc.columns.tolist()
df_cols.sort()
desired_cols = ['team', 'ratio']
desired_cols.sort()

assert win_perc.shape[0] == 5, "Expected 5 rows, got {}".format(win_perc.shape[0])
assert win_perc.shape[1] == 2, "Expected 2 columns, got {}".format(win_perc.shape[1])
assert df_cols == desired_cols, "Expected {} columns but got {} columns".format(desired_cols, df_cols)

tolerance = .001
sec_team = win_perc.iloc[1].team
sec_perc = win_perc.iloc[1].ratio

assert (sec_team == "Brazil" and abs(sec_perc - .773) <= tolerance), "Second team should be {} with ratio of {}. \
Got {} with ratio of {}".format("Brazil", .773, sec_team, sec_perc)

print("\n(Passed!)")

The solution, according to you:


,team,ratio
0,Spain,0.800
1,Brazil,0.773
2,Iran,0.742
3,Cameroon,0.739
4,Egypt,0.724



(Passed!)


In [19]:
# Hidden test cell: exercise4_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Exercise 5** (3 points) Now, we've seen how much the home advantage plays in, let us see how the results have looked 
like in the previous tournaments, for the specific case of the FIFA World Cup matches.

In particular, complete the function, `points_table(winners_df, wc_year)`, below, so that it does the following:
- It should take as input a dataframe, `winners_df`, having a "winner" column like that produced in Exercise 3, as well as a target year, `wc_year`.
- It should consider only games in the given target year. Furthermore, it should only consider games where the `tournament` column has the value `"FIFA World Cup"`.
- It should construct and return a "points table". This table should have two columns, **`team`**, containing the team name, and **`points`**, containing a points tally has defined below.
- To compute the points, give the team 3 points for every win, 1 point for every draw, and 0 points (no points) for a loss.
- In case of a tie in the points, sort the teams alphabetically

As an example output, for the 1998 FIFA World Cup, the points table is:

| team        | points |
|-------------|--------|
| France      | 19     |
| Croatia     | 15     |
| Brazil      | 13     |
| Netherlands | 12     |
| Italy       | 11     |

In [36]:
def points_table(winners_df, wc_year):
    ###
    ### YOUR CODE HERE
    ###
    import datetime
    winners_df = determine_winners(winners_df)
    winners_df['date'] = pd.to_datetime(winners_df['date'])
    winners_df['year'] = winners_df['date'].dt.strftime('%Y')
    winners_df = winners_df.loc[(winners_df['tournament']=='FIFA World Cup') & (winners_df['year']==str(wc_year))]
    df_5 = pd.concat([winners_df[['home_team','winner']].rename(columns={'home_team':'team'}),winners_df[['away_team','winner']].rename(columns={'away_team':'team'})],axis=0)
    df_5['points']=df_5.apply(lambda x: 3 if x['team']==x['winner'] else (1 if x['winner']=='Draw' else 0),axis=1)
    df_5 = df_5.groupby('team')['points'].agg('sum').reset_index().sort_values(['points','team'], ascending=[False,True])
    return df_5.head()

In [35]:
points_table(winners_df, 1998)

,team,points
11,France,19
8,Croatia,15
3,Brazil,13
19,Netherlands,12
14,Italy,11
0,Argentina,10
12,Germany,10
9,Denmark,7
10,England,7
23,Romania,7


In [26]:
# Test: Exercise 5 (exposed)


tbl_1998 = points_table(winners_df, 1998)

assert tbl_1998.iloc[0].team == "France"
assert tbl_1998.iloc[0].points == 19
assert tbl_1998.iloc[1].team == "Croatia"
assert tbl_1998.iloc[1].points == 15
assert tbl_1998.iloc[2].team == "Brazil"
assert tbl_1998.iloc[2].points == 13
assert tbl_1998.iloc[3].team == "Netherlands"
assert tbl_1998.iloc[3].points == 12
assert tbl_1998.iloc[4].team == "Italy"
assert tbl_1998.iloc[4].points == 11

print("\n(Passed!)")



(Passed!)


In [27]:
# Hidden test cell: exercise5_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Fin!** You’ve reached the end of this part. Don’t forget to restart and run all cells again to make sure it’s all working when run in sequence; and make sure your work passes the submission process. Good luck!